In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd
import logging

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions


In [4]:
from pydrake.geometry.optimization_dev import CspaceFreePolytope, FindPolytopeGivenLagrangianOptions, FindSeparationCertificateGivenPolytopeOptions, BilinearAlternationOptions, SeparatingPlaneOrder

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))




locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()

idx = 0
q0 = [0.0, 0.0]
# q_low  = [-1.7, -1.7, 0]
# q_high = [1.7, 1.7,  0]
eps = 0# 1e-3
val = 1.7
q_low  = [-val+eps, -val+eps, 0]
q_high = [val-eps, val-eps,  0]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_s = []
for q in [q_low, q_high]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star)[:2])

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001
/home/amice/Documents/coding_projects/drake/C_Iris_Examples/iris_plant_visualizer.py:70: DrakeDeprecationWarning: (Deprecated) See ForcedPublish() 
*/ (Deprecated.)

Deprecated:
    Use ForcedPublish() instead This will be removed from Drake on or
    after 2023-03-01.
  self.diagram.Publish(self.diagram_context)


In [6]:
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))


## Set up the sliders so we can move the plant around manually

In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [9]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

right_sweeper_fused_col_geom = right_sweeper_gids[2:]
right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
left_sweeper_fused_col_geom = left_sweeper_gids[4:]
left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

right_sweeper_end_gid = right_sweeper_gids[-1]
left_sweeper_end_gid = left_sweeper_gids[-1]
id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
                       ]
visualizer.collision_pairs_of_interest = id_pairs_of_interest


# Setup IRIS Options and Generate Regions

In [10]:
# Some examples of some seed points which give large regions using the
# non-linear programming approach to IRIS described in Algorithm 3
seed_points_q = np.array([   [0.0, 0],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.4,-1.3]
                              ])
seed_points = np.array([Ratfk.ComputeSValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)


In [11]:
regions = []
ellipses = []

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.configuration_space_margin = 1e-3
iris_options.relative_termination_threshold = 0.001

def promote_region_to_3d(region, width = 0.2):
    A = block_diag(region.A(), np.array([-1,1])[:, np.newaxis])
    b = np.append(region.b(), width*np.ones(2))

    return HPolyhedron(A,b)
    
regions_3d = []

for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, np.zeros((2,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q)
    r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), q_star, iris_options)
    regions.append(r)
    regions_3d.append(promote_region_to_3d(r))
    ellipses.append(r.MaximumVolumeInscribedEllipsoid())
    
# chosen for colorblind compatability
colors_dict = {
    0: (144,144,144),
    1:(30,136,229), # bluish
    2: (255, 193, 7), # gold
    3: (0, 140, 6), # green
    
    
}
if do_viz:
    visualizer.plot_regions(regions,
                            ellipses=ellipses,
                            region_suffix='_original',
                            colors = list(colors_dict.values()),
                            wireframe = False,
                           opacity = 0.2)

# Certify regions

In [12]:
log = logging.getLogger("drake")
log.setLevel("DEBUG")

In [13]:
cspace_free_polytope = CspaceFreePolytope(plant, scene_graph, SeparatingPlaneOrder.kAffine, q_star)


DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will us

DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will us

DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will us

In [14]:
# options
solver_options = SolverOptions()
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 0)

find_polytope_given_lagrangian_option = FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.s_inner_pts = np.zeros(2)

find_separation_certificate_given_polytope_options = FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.num_threads = -1
find_separation_certificate_given_polytope_options.verbose = True
find_separation_certificate_given_polytope_options.solver_options = solver_options

bilinear_alternation_options = BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 10
bilinear_alternation_options.convergence_tol = 1e-5
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option
bilinear_alternation_options.find_lagrangian_options = find_separation_certificate_given_polytope_options


In [15]:
C_init = np.vstack([np.eye(2), -np.eye(2)])
d_init = 1e-2* np.ones(C_init.shape[0])
test_region = HPolyhedron(C_init, d_init)
visualizer.plot_regions([test_region],
                            region_suffix='_test',
                            wireframe = False,
                           opacity = 0.5)


DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek


In [16]:
(success, certificates) = cspace_free_polytope.FindSeparationCertificateGivenPolytope(C=C_init, d = d_init,
                                                                  ignored_collision_pairs = set(),
                                                                  search_separating_margin = False,
                                                                  options = find_separation_certificate_given_polytope_options)
print(success)

DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will us

DEBUG:drake:SOS 89/130 dispatched
DEBUG:drake:SOS 90/130 dispatched
DEBUG:drake:SOS 91/130 dispatched
DEBUG:drake:SOS 92/130 dispatched
DEBUG:drake:SOS 70/130 completed, is_success true
DEBUG:drake:SOS 71/130 completed, is_success true
DEBUG:drake:SOS 72/130 completed, is_success true
DEBUG:drake:SOS 73/130 completed, is_success true
DEBUG:drake:SOS 74/130 completed, is_success true
DEBUG:drake:SOS 75/130 completed, is_success true
DEBUG:drake:SOS 76/130 completed, is_success true
DEBUG:drake:SOS 77/130 completed, is_success true
DEBUG:drake:SOS 78/130 completed, is_success true
DEBUG:drake:SOS 92/130 completed, is_success true
DEBUG:drake:SOS 93/130 dispatched
DEBUG:drake:SOS 94/130 dispatched
DEBUG:drake:SOS 95/130 dispatched
DEBUG:drake:SOS 96/130 dispatched
DEBUG:drake:SOS 97/130 dispatched
DEBUG:drake:SOS 98/130 dispatched
DEBUG:drake:SOS 99/130 dispatched
DEBUG:drake:SOS 100/130 dispatched
DEBUG:drake:SOS 101/130 dispatched
DEBUG:drake:SOS 102/130 dispatched
DEBUG:drake:SOS 86/13

True


In [17]:
(success, certificates) = cspace_free_polytope.FindSeparationCertificateGivenPolytope(C=C_init, d = d_init,
                                                                  ignored_collision_pairs = set(),
                                                                  search_separating_margin = True,
                                                                  options = find_separation_certificate_given_polytope_options)
print(success)

DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will us

False


In [ ]:
result = cspace_free_polytope.SearchWithBilinearAlternation(ignored_collision_pairs = set(),
                                                                    C_init=C_init, d_init=d_init,
                                                                    search_margin=False,
                                                                    options=bilinear_alternation_options)

DEBUG:drake:solvers::Solve will use Mosek
INFO:drake:det(Q) at the beginning is 0.00010000001401025257
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:SOS 0/130 dispatched
DEBUG:drake:SOS 1/130 dispatched
DEBUG:drake:SOS 2/130 dispatched
DEBUG:drake:SOS 3/130 dispatched
DEBUG:drake:SOS 4/130 dispatched
DEBUG:drake:SOS 5/130 dispatched
DEBUG:drake:SOS 6/130 dispa

DEBUG:drake:SOS 94/130 dispatched
DEBUG:drake:SOS 95/130 dispatched
DEBUG:drake:SOS 96/130 dispatched
DEBUG:drake:SOS 97/130 dispatched
DEBUG:drake:SOS 98/130 dispatched
DEBUG:drake:SOS 99/130 dispatched
DEBUG:drake:SOS 100/130 dispatched
DEBUG:drake:SOS 81/130 completed, is_success true
DEBUG:drake:SOS 91/130 completed, is_success true
DEBUG:drake:SOS 94/130 completed, is_success true
DEBUG:drake:SOS 95/130 completed, is_success true
DEBUG:drake:SOS 96/130 completed, is_success true
DEBUG:drake:SOS 97/130 completed, is_success true
DEBUG:drake:SOS 101/130 dispatched
DEBUG:drake:SOS 102/130 dispatched
DEBUG:drake:SOS 103/130 dispatched
DEBUG:drake:SOS 104/130 dispatched
DEBUG:drake:SOS 105/130 dispatched
DEBUG:drake:SOS 106/130 dispatched
DEBUG:drake:SOS 86/130 completed, is_success true
DEBUG:drake:SOS 87/130 completed, is_success true
DEBUG:drake:SOS 88/130 completed, is_success true
DEBUG:drake:SOS 89/130 completed, is_success true
DEBUG:drake:SOS 90/130 completed, is_success true
D

DEBUG:drake:SOS 51/130 completed, is_success true
DEBUG:drake:SOS 52/130 completed, is_success true
DEBUG:drake:SOS 54/130 completed, is_success true
DEBUG:drake:SOS 55/130 dispatched
DEBUG:drake:SOS 56/130 dispatched
DEBUG:drake:SOS 57/130 dispatched
DEBUG:drake:SOS 58/130 dispatched
DEBUG:drake:SOS 39/130 completed, is_success true
DEBUG:drake:SOS 40/130 completed, is_success true
DEBUG:drake:SOS 41/130 completed, is_success true
DEBUG:drake:SOS 42/130 completed, is_success true
DEBUG:drake:SOS 43/130 completed, is_success true
DEBUG:drake:SOS 44/130 completed, is_success true
DEBUG:drake:SOS 45/130 completed, is_success true
DEBUG:drake:SOS 46/130 completed, is_success true
DEBUG:drake:SOS 47/130 completed, is_success true
DEBUG:drake:SOS 48/130 completed, is_success true
DEBUG:drake:SOS 49/130 completed, is_success true
DEBUG:drake:SOS 53/130 completed, is_success true
DEBUG:drake:SOS 55/130 completed, is_success true
DEBUG:drake:SOS 56/130 completed, is_success true
DEBUG:drake:SO

DEBUG:drake:SOS 8/130 dispatched
DEBUG:drake:SOS 9/130 dispatched
DEBUG:drake:SOS 10/130 dispatched
DEBUG:drake:SOS 11/130 dispatched
DEBUG:drake:SOS 12/130 dispatched
DEBUG:drake:SOS 13/130 dispatched
DEBUG:drake:SOS 14/130 dispatched
DEBUG:drake:SOS 15/130 dispatched
DEBUG:drake:SOS 3/130 completed, is_success true
DEBUG:drake:SOS 4/130 completed, is_success true
DEBUG:drake:SOS 5/130 completed, is_success true
DEBUG:drake:SOS 6/130 completed, is_success true
DEBUG:drake:SOS 7/130 completed, is_success true
DEBUG:drake:SOS 8/130 completed, is_success true
DEBUG:drake:SOS 9/130 completed, is_success true
DEBUG:drake:SOS 10/130 completed, is_success true
DEBUG:drake:SOS 11/130 completed, is_success true
DEBUG:drake:SOS 12/130 completed, is_success true
DEBUG:drake:SOS 13/130 completed, is_success true
DEBUG:drake:SOS 14/130 completed, is_success true
DEBUG:drake:SOS 15/130 completed, is_success true
DEBUG:drake:SOS 16/130 dispatched
DEBUG:drake:SOS 17/130 dispatched
DEBUG:drake:SOS 18/

DEBUG:drake:SOS 92/130 completed, is_success true
DEBUG:drake:SOS 109/130 dispatched
DEBUG:drake:SOS 110/130 dispatched
DEBUG:drake:SOS 111/130 dispatched
DEBUG:drake:SOS 112/130 dispatched
DEBUG:drake:SOS 99/130 completed, is_success true
DEBUG:drake:SOS 100/130 completed, is_success true
DEBUG:drake:SOS 101/130 completed, is_success true
DEBUG:drake:SOS 102/130 completed, is_success true
DEBUG:drake:SOS 103/130 completed, is_success true
DEBUG:drake:SOS 104/130 completed, is_success true
DEBUG:drake:SOS 105/130 completed, is_success true
DEBUG:drake:SOS 106/130 completed, is_success true
DEBUG:drake:SOS 107/130 completed, is_success true
DEBUG:drake:SOS 108/130 completed, is_success true
DEBUG:drake:SOS 109/130 completed, is_success true
DEBUG:drake:SOS 110/130 completed, is_success true
DEBUG:drake:SOS 111/130 completed, is_success true
DEBUG:drake:SOS 112/130 completed, is_success true
DEBUG:drake:SOS 113/130 dispatched
DEBUG:drake:SOS 114/130 dispatched
DEBUG:drake:SOS 115/130 dis

...

DEBUG:drake:SOS 112/130 completed, is_success true
DEBUG:drake:SOS 113/130 completed, is_success true
DEBUG:drake:SOS 114/130 completed, is_success true
DEBUG:drake:SOS 115/130 completed, is_success true
DEBUG:drake:SOS 116/130 dispatched
DEBUG:drake:SOS 117/130 dispatched
DEBUG:drake:SOS 118/130 dispatched
DEBUG:drake:SOS 119/130 dispatched
DEBUG:drake:SOS 120/130 dispatched
DEBUG:drake:SOS 121/130 dispatched
DEBUG:drake:SOS 122/130 dispatched
DEBUG:drake:SOS 123/130 dispatched
DEBUG:drake:SOS 124/130 dispatched
DEBUG:drake:SOS 125/130 dispatched
DEBUG:drake:SOS 126/130 dispatched
DEBUG:drake:SOS 127/130 dispatched
DEBUG:drake:SOS 128/130 dispatched
DEBUG:drake:SOS 129/130 dispatched
DEBUG:drake:SOS 120/130 completed, is_success true
DEBUG:drake:SOS 121/130 completed, is_success true
DEBUG:drake:SOS 122/130 completed, is_success true
DEBUG:drake:SOS 123/130 completed, is_success true
DEBUG:drake:SOS 124/130 completed, is_success true
DEBUG:drake:SOS 125/130 completed, is_success true


DEBUG:drake:SOS 76/130 dispatched
DEBUG:drake:SOS 77/130 dispatched
DEBUG:drake:SOS 78/130 dispatched
DEBUG:drake:SOS 79/130 dispatched
DEBUG:drake:SOS 80/130 dispatched
DEBUG:drake:SOS 81/130 dispatched
DEBUG:drake:SOS 82/130 dispatched
DEBUG:drake:SOS 83/130 dispatched
DEBUG:drake:SOS 79/130 completed, is_success true
DEBUG:drake:SOS 80/130 completed, is_success true
DEBUG:drake:SOS 82/130 completed, is_success true
DEBUG:drake:SOS 83/130 completed, is_success true
DEBUG:drake:SOS 84/130 dispatched
DEBUG:drake:SOS 85/130 dispatched
DEBUG:drake:SOS 86/130 dispatched
DEBUG:drake:SOS 87/130 dispatched
DEBUG:drake:SOS 68/130 completed, is_success true
DEBUG:drake:SOS 69/130 completed, is_success true
DEBUG:drake:SOS 70/130 completed, is_success true
DEBUG:drake:SOS 71/130 completed, is_success true
DEBUG:drake:SOS 72/130 completed, is_success true
DEBUG:drake:SOS 73/130 completed, is_success true
DEBUG:drake:SOS 74/130 completed, is_success true
DEBUG:drake:SOS 75/130 completed, is_succe

DEBUG:drake:SOS 34/130 dispatched
DEBUG:drake:SOS 35/130 dispatched
DEBUG:drake:SOS 36/130 dispatched
DEBUG:drake:SOS 37/130 dispatched
DEBUG:drake:SOS 38/130 dispatched
DEBUG:drake:SOS 39/130 dispatched
DEBUG:drake:SOS 40/130 dispatched
DEBUG:drake:SOS 16/130 completed, is_success true
DEBUG:drake:SOS 17/130 completed, is_success true
DEBUG:drake:SOS 18/130 completed, is_success true
DEBUG:drake:SOS 19/130 completed, is_success true
DEBUG:drake:SOS 21/130 completed, is_success true
DEBUG:drake:SOS 41/130 dispatched
DEBUG:drake:SOS 42/130 dispatched
DEBUG:drake:SOS 43/130 dispatched
DEBUG:drake:SOS 44/130 dispatched
DEBUG:drake:SOS 45/130 dispatched
DEBUG:drake:SOS 30/130 completed, is_success true
DEBUG:drake:SOS 31/130 completed, is_success true
DEBUG:drake:SOS 32/130 completed, is_success true
DEBUG:drake:SOS 33/130 completed, is_success true
DEBUG:drake:SOS 34/130 completed, is_success true
DEBUG:drake:SOS 35/130 completed, is_success true
DEBUG:drake:SOS 36/130 completed, is_succe

DEBUG:drake:SOS 126/130 completed, is_success true
DEBUG:drake:SOS 127/130 completed, is_success true
DEBUG:drake:SOS 128/130 completed, is_success true
DEBUG:drake:SOS 129/130 completed, is_success true
DEBUG:drake:SOS 120/130 completed, is_success true
DEBUG:drake:SOS 121/130 completed, is_success true
DEBUG:drake:SOS 122/130 completed, is_success true
INFO:drake:Found Lagrangian multipliers and separating planes
DEBUG:drake:solvers::Solve will use Mosek
INFO:drake:Iteration 6: det(Q)=0.001549866085533523
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:solvers::Solv

DEBUG:drake:SOS 84/130 dispatched
DEBUG:drake:SOS 85/130 dispatched
DEBUG:drake:SOS 86/130 dispatched
DEBUG:drake:SOS 87/130 dispatched
DEBUG:drake:SOS 88/130 dispatched
DEBUG:drake:SOS 89/130 dispatched
DEBUG:drake:SOS 90/130 dispatched
DEBUG:drake:SOS 91/130 dispatched
DEBUG:drake:SOS 92/130 dispatched
DEBUG:drake:SOS 93/130 dispatched
DEBUG:drake:SOS 94/130 dispatched
DEBUG:drake:SOS 95/130 dispatched
DEBUG:drake:SOS 96/130 dispatched
DEBUG:drake:SOS 97/130 dispatched
DEBUG:drake:SOS 98/130 dispatched
DEBUG:drake:SOS 99/130 dispatched
DEBUG:drake:SOS 94/130 completed, is_success true
DEBUG:drake:SOS 95/130 completed, is_success true
DEBUG:drake:SOS 97/130 completed, is_success true
DEBUG:drake:SOS 98/130 completed, is_success true
DEBUG:drake:SOS 99/130 completed, is_success true
DEBUG:drake:SOS 100/130 dispatched
DEBUG:drake:SOS 101/130 dispatched
DEBUG:drake:SOS 102/130 dispatched
DEBUG:drake:SOS 103/130 dispatched
DEBUG:drake:SOS 104/130 dispatched
DEBUG:drake:SOS 84/130 complete

In [ ]:
grown_regions = HPolyhedron(result.C, result.d)
visualizer.plot_regions([test_region],
                            region_suffix='_grown',
                            wireframe = False,
                           opacity = 0.5)

In [ ]:
bilinear_alternation_options.max_iter
